In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import calendar
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
!pip install keras_tqdm


In [0]:
import numpy
import pandas
import matplotlib.pyplot as plt
import subprocess
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import csv
import sklearn
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping

In [0]:
trainingOrderItems = pandas.read_csv('/content/drive/My Drive/data/InstaCart Market Basket Analysis/Dataset for InstaCart/order_products__train.csv')
priorOrderItems = pandas.read_csv('/content/drive/My Drive/data/InstaCart Market Basket Analysis/Dataset for InstaCart/order_products__prior.csv')
orders = pandas.read_csv('/content/drive/My Drive/data/InstaCart Market Basket Analysis/Dataset for InstaCart/orders.csv')
products = pandas.read_csv('/content/drive/My Drive/data/InstaCart Market Basket Analysis/Dataset for InstaCart/products.csv')

In [0]:
productGroup = priorOrderItems.groupby("product_id")
productReorders = productGroup['order_id'].aggregate(numpy.size).to_frame()
productReorders['reordered'] = productGroup['reordered'].aggregate(numpy.mean)
del productGroup

In [37]:
orderGroup = priorOrderItems.groupby("order_id").aggregate(numpy.max)['add_to_cart_order'].to_frame()
priorOrderItems = pandas.merge(priorOrderItems,orderGroup.rename(columns={'add_to_cart_order':'basket_size'}),left_on='order_id', right_index=True)
priorOrderItems.head()

,order_id,product_id,add_to_cart_order,reordered,basket_size
0,2,33120,1,1,9
1,2,28985,2,1,9
2,2,9327,3,0,9
3,2,45918,4,1,9
4,2,30035,5,0,9


In [0]:
orderItems = pandas.merge(orders,priorOrderItems, on='order_id')
orderItems['user_product_id'] = 1000000*orderItems['user_id']+orderItems['product_id']
userProducts = orderItems[orderItems['eval_set']=='prior']
userProducts = userProducts[['user_id','order_dow','order_hour_of_day','days_since_prior_order',
                           'product_id','add_to_cart_order', 'user_product_id', 'basket_size', 'order_number', 'order_id']]
userProducts['orders'] = userProducts['user_id']
userAvgProducts = userProducts.groupby('user_product_id').aggregate({'user_id':'mean','order_dow':'mean','order_hour_of_day':'mean',
                                                   'days_since_prior_order':'mean', 'orders':'count',
                                                   'product_id':'mean','add_to_cart_order':'mean', 'basket_size':'mean'})
del orderItems

In [39]:
lastOrderId = userProducts[userProducts.groupby(['user_product_id'])['order_number'].transform(max) == userProducts['order_number']]
lastOrderId = lastOrderId[['user_product_id','order_id']]
lastOrderId.rename(columns={'order_id':'last_order_id'}, inplace=True)
userAvgProducts = pandas.merge(userAvgProducts, lastOrderId, right_on='user_product_id', left_index=True)
userAvgProducts.set_index('user_product_id', inplace=True)
userAvgProducts.head()

,user_id,order_dow,order_hour_of_day,days_since_prior_order,orders,product_id,add_to_cart_order,basket_size,last_order_id
user_product_id,,,,,,,,,
1000196,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362
1010258,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362
1010326,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534
1012427,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362
1013032,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362


In [0]:
user_distinct_products = userAvgProducts.groupby('user_id').count()['product_id'].to_frame()
user_distinct_products['user_reordered_products'] = userAvgProducts[userAvgProducts['orders']>1].groupby('user_id').count()['product_id']

In [41]:

user_distinct_products.rename(columns={'product_id':'distinct_products'},inplace=True)
user_distinct_products.head()

,distinct_products,user_reordered_products
user_id,,
1,18,10.0
2,102,37.0
3,33,19.0
4,17,1.0
5,23,8.0


In [42]:
userAvgProducts = pandas.merge(user_distinct_products,userAvgProducts,left_index=True,right_on='user_id')
userAvgProducts.head()

,distinct_products,user_reordered_products,user_id,order_dow,order_hour_of_day,days_since_prior_order,orders,product_id,add_to_cart_order,basket_size,last_order_id
user_product_id,,,,,,,,,,,
1000196,18,10.0,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362
1010258,18,10.0,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362
1010326,18,10.0,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534
1012427,18,10.0,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362
1013032,18,10.0,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362


In [43]:
userAvgProducts['user_reorder_rate'] = userAvgProducts['user_reordered_products']/userAvgProducts['distinct_products']
userAvgProducts.head()

,distinct_products,user_reordered_products,user_id,order_dow,order_hour_of_day,days_since_prior_order,orders,product_id,add_to_cart_order,basket_size,last_order_id,user_reorder_rate
user_product_id,,,,,,,,,,,,
1000196,18,10.0,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1010258,18,10.0,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
1010326,18,10.0,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
1012427,18,10.0,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
1013032,18,10.0,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [0]:
features = userAvgProducts
features.columns = ['user_distinct_products','user_reordered_products','user_id','user_product_dow','user_product_hod','user_product_dsp','user_product_orders','product_id','user_product_addCart', 'user_avg_basket_size', 'user_product_last_order_id','user_reorder_rate']

In [45]:
userOrders = orders[orders['eval_set']=='prior'].groupby('user_id').aggregate(numpy.max)
userOrders.head()

,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
user_id,,,,,,
1,3367565,prior,10,4,16,30.0
2,3268552,prior,14,5,15,30.0
3,3225766,prior,12,3,19,21.0
4,3343014,prior,5,6,15,21.0
5,2717275,prior,4,3,18,19.0


In [46]:
features.head()

,user_distinct_products,user_reordered_products,user_id,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate
user_product_id,,,,,,,,,,,,
1000196,18,10.0,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1010258,18,10.0,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
1010326,18,10.0,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
1012427,18,10.0,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
1013032,18,10.0,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [47]:
userOrders=userOrders[['order_number']]
features = pandas.merge(userOrders,features, right_on='user_id', left_index=True)
features.head()

,order_number,user_distinct_products,user_reordered_products,user_id,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate
user_product_id,,,,,,,,,,,,,
1000196,10,18,10.0,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1010258,10,18,10.0,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
1010326,10,18,10.0,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
1012427,10,18,10.0,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
1013032,10,18,10.0,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [48]:
features['order_number'] = features['user_product_orders']/features['order_number']
features.head(5)

,order_number,user_distinct_products,user_reordered_products,user_id,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate
user_product_id,,,,,,,,,,,,,
1000196,1.0,18,10.0,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1010258,0.9,18,10.0,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
1010326,0.1,18,10.0,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
1012427,1.0,18,10.0,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
1013032,0.3,18,10.0,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [49]:
features.rename(columns={'order_number': 'user_product_reorder_rate'}, inplace=True)
features.head()

,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_id,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate
user_product_id,,,,,,,,,,,,,
1000196,1.0,18,10.0,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1010258,0.9,18,10.0,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
1010326,0.1,18,10.0,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
1012427,1.0,18,10.0,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
1013032,0.3,18,10.0,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [50]:
features=pandas.merge(features,productReorders, left_on='product_id',right_index=True)
features.rename(columns={'order_id':'product_total_orders','reordered':'product_reorder_rate'},inplace=True)
features['product_reorders'] = features['product_reorder_rate'] * features['product_total_orders']
features.head()

,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_id,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders
user_product_id,,,,,,,,,,,,,,,,
1000196,1.000000,18,10.0,1,2.5,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556,35791,0.77648,27791.0
15000196,0.227273,13,9.0,15,1.4,11.000000,8.600000,5,196,2.200000,2.800000,487368,0.692308,35791,0.77648,27791.0
19000196,0.333333,133,39.0,19,5.0,14.333333,6.666667,3,196,6.333333,31.333333,532817,0.293233,35791,0.77648,27791.0
21000196,0.030303,102,30.0,21,3.0,10.000000,6.000000,1,196,2.000000,7.000000,1573906,0.294118,35791,0.77648,27791.0
31000196,0.100000,190,66.0,31,3.0,9.500000,7.500000,2,196,15.500000,27.500000,2231262,0.347368,35791,0.77648,27791.0


In [51]:

features = pandas.merge(features, products, on='product_id')
features.drop('product_name',axis=1,inplace=True)
features.head()

,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_id,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,aisle_id,department_id
0,1.000000,18,10.0,1,2.5,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556,35791,0.77648,27791.0,77,7
1,0.227273,13,9.0,15,1.4,11.000000,8.600000,5,196,2.200000,2.800000,487368,0.692308,35791,0.77648,27791.0,77,7
2,0.333333,133,39.0,19,5.0,14.333333,6.666667,3,196,6.333333,31.333333,532817,0.293233,35791,0.77648,27791.0,77,7
3,0.030303,102,30.0,21,3.0,10.000000,6.000000,1,196,2.000000,7.000000,1573906,0.294118,35791,0.77648,27791.0,77,7
4,0.100000,190,66.0,31,3.0,9.500000,7.500000,2,196,15.500000,27.500000,2231262,0.347368,35791,0.77648,27791.0,77,7


In [56]:
# Ready the inputs into lightGBM
train_orders = orders[orders['eval_set']=='train']
test_orders = orders[orders['eval_set']=='test']

test_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [0]:
train_features = pandas.merge(train_orders, features, on='user_id')
test_features = pandas.merge(test_orders, features, on='user_id')

In [58]:
test_features.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders_x,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,aisle_id,department_id,"(user_orders,)","('user_order_dsp',)_x","('user_product_orders',)_x","(user_order_dsp_y,)","(user_product_orders_y,)","('user_order_dsp',)_x","(user_product_orders,)","(user_order_dsp_y,)"
0,2774568,3,test,13,5,15,11.0,0.250000,33,19.0,0.333333,15.000000,7.000000,3,16797,4.000000,8.000000,3160850,0.575758,142951,0.698155,99802.0,24,4,12,12.090909,12,12.090909,12,12.090909,12,12.090909
1,2774568,3,test,13,5,15,11.0,0.750000,33,19.0,0.888889,16.111111,12.125000,9,47766,3.777778,7.222222,1402502,0.575758,176815,0.758103,134044.0,24,4,12,12.090909,12,12.090909,12,12.090909,12,12.090909
2,2774568,3,test,13,5,15,11.0,0.083333,33,19.0,3.000000,19.000000,9.000000,1,248,3.000000,9.000000,444309,0.575758,6371,0.400251,2550.0,117,19,12,12.090909,12,12.090909,12,12.090909,12,12.090909
3,2774568,3,test,13,5,15,11.0,0.083333,33,19.0,3.000000,16.000000,17.000000,1,1005,5.000000,6.000000,676467,0.575758,463,0.440605,204.0,94,7,12,12.090909,12,12.090909,12,12.090909,12,12.090909
4,2774568,3,test,13,5,15,11.0,0.250000,33,19.0,0.666667,16.333333,11.333333,3,1819,2.666667,7.333333,1839752,0.575758,2424,0.492162,1193.0,88,13,12,12.090909,12,12.090909,12,12.090909,12,12.090909


In [0]:

del train_orders
del test_orders
del features

In [62]:

train_features['diff_order_hod'] = abs(train_features['user_product_hod']-train_features['order_hour_of_day']).map(lambda x: min(x, 24-x))
train_features['ratio_dsp'] = train_features['user_product_dsp']/train_features['days_since_prior_order']
train_features['diff_dow'] = abs(train_features['user_product_dow']-train_features['order_dow']).map(lambda x: min (x, 7-x))
#train_features['user_product_orders_since_last'] = train_features['user_orders'] - train_features['user_product_last_order_id'].map(orders.order_number)
train_features['user_product_hour_vs_last'] = abs(train_features['order_hour_of_day'] - train_features['user_product_last_order_id'].map(orders.order_hour_of_day) \
                                                 ).map(lambda x: min(x, 24-x))

test_features['diff_order_hod'] = abs(test_features['user_product_hod']-test_features['order_hour_of_day']).map(lambda x: min(x, 24-x))
test_features['ratio_dsp'] = test_features['user_product_dsp']/test_features['days_since_prior_order']
test_features['diff_dow'] = abs(test_features['user_product_dow']-test_features['order_dow']).map(lambda x: min (x, 7-x))
#test_features['user_product_orders_since_last'] = test_features['user_orders'] - test_features['user_product_last_order_id'].map(orders.order_number)
test_features['user_product_hour_vs_last'] = abs(test_features['order_hour_of_day'] - test_features['user_product_last_order_id'].map(orders.order_hour_of_day) \
                                                 ).map(lambda x: min(x, 24-x))

test_features.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders_x,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,aisle_id,department_id,"(user_orders,)","('user_order_dsp',)_x","('user_product_orders',)_x","(user_order_dsp_y,)","(user_product_orders_y,)","('user_order_dsp',)_x","(user_product_orders,)","(user_order_dsp_y,)",diff_order_hod,ratio_dsp,diff_dow,user_product_hour_vs_last
0,2774568,3,test,13,5,15,11.0,0.250000,33,19.0,0.333333,15.000000,7.000000,3,16797,4.000000,8.000000,3160850,0.575758,142951,0.698155,99802.0,24,4,12,12.090909,12,12.090909,12,12.090909,12,12.090909,0.000000,0.636364,2.333333,1.0
1,2774568,3,test,13,5,15,11.0,0.750000,33,19.0,0.888889,16.111111,12.125000,9,47766,3.777778,7.222222,1402502,0.575758,176815,0.758103,134044.0,24,4,12,12.090909,12,12.090909,12,12.090909,12,12.090909,1.111111,1.102273,2.888889,0.0
2,2774568,3,test,13,5,15,11.0,0.083333,33,19.0,3.000000,19.000000,9.000000,1,248,3.000000,9.000000,444309,0.575758,6371,0.400251,2550.0,117,19,12,12.090909,12,12.090909,12,12.090909,12,12.090909,4.000000,0.818182,2.000000,4.0
3,2774568,3,test,13,5,15,11.0,0.083333,33,19.0,3.000000,16.000000,17.000000,1,1005,5.000000,6.000000,676467,0.575758,463,0.440605,204.0,94,7,12,12.090909,12,12.090909,12,12.090909,12,12.090909,1.000000,1.545455,2.000000,2.0
4,2774568,3,test,13,5,15,11.0,0.250000,33,19.0,0.666667,16.333333,11.333333,3,1819,2.666667,7.333333,1839752,0.575758,2424,0.492162,1193.0,88,13,12,12.090909,12,12.090909,12,12.090909,12,12.090909,1.333333,1.030303,2.666667,9.0


In [0]:
train_features.sort_values(['order_id','product_id'],inplace=True)

In [64]:
test_features.sort_values(['order_id','product_id'],inplace=True)
test_features.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders_x,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,aisle_id,department_id,"(user_orders,)","('user_order_dsp',)_x","('user_product_orders',)_x","(user_order_dsp_y,)","(user_product_orders_y,)","('user_order_dsp',)_x","(user_product_orders,)","(user_order_dsp_y,)",diff_order_hod,ratio_dsp,diff_dow,user_product_hour_vs_last
858095,17,36855,test,5,6,15,1.0,0.25,21,5.0,0.000000,9.0,30.0,1,1283,5.000000,9.000000,234692,0.238095,29,0.206897,6.0,100,21,4,14.0,4,14.0,4,14.0,4,14.0,6.0,30.0,1.000000,3.0
858092,17,36855,test,5,6,15,1.0,0.25,21,5.0,6.000000,14.0,6.0,1,6291,9.000000,9.000000,3248434,0.238095,1968,0.618394,1217.0,121,14,4,14.0,4,14.0,4,14.0,4,14.0,1.0,6.0,0.000000,2.0
858088,17,36855,test,5,6,15,1.0,0.25,21,5.0,1.000000,13.0,NaN,1,7035,3.000000,5.000000,898818,0.238095,1933,0.478013,924.0,100,21,4,14.0,4,14.0,4,14.0,4,14.0,2.0,NaN,2.000000,2.0
858094,17,36855,test,5,6,15,1.0,0.25,21,5.0,1.000000,13.0,NaN,1,11494,5.000000,5.000000,898818,0.238095,1311,0.363082,476.0,100,21,4,14.0,4,14.0,4,14.0,4,14.0,2.0,NaN,2.000000,2.0
858097,17,36855,test,5,6,15,1.0,0.75,21,5.0,3.666667,12.0,14.0,3,13107,2.333333,7.333333,1058761,0.238095,373,0.635389,237.0,3,19,4,14.0,4,14.0,4,14.0,4,14.0,3.0,14.0,2.333333,3.0


In [0]:
train_features.drop(['eval_set','order_number','order_number'],axis=1,inplace=True)


In [66]:
test_features.drop(['eval_set','order_number','order_number'],axis=1,inplace=True)
train_features.head()

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders_x,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,aisle_id,department_id,"(user_orders,)","('user_order_dsp',)_x","('user_product_orders',)_x","(user_order_dsp_y,)","(user_product_orders_y,)","('user_order_dsp',)_x","(user_product_orders,)","(user_order_dsp_y,)",diff_order_hod,ratio_dsp,diff_dow,user_product_hour_vs_last
4629306,1,112108,4,10,9.0,0.333333,12,8.0,1.000000,10.0,7.0,1,2067,7.000000,8.0,2541372,0.666667,3138,0.606119,1902.0,3,19,3,11.0,3,11.0,3,11.0,3,11.0,0.0,0.777778,3.000000,1
4629308,1,112108,4,10,9.0,0.666667,12,8.0,1.500000,15.5,11.0,2,5707,5.000000,6.5,186706,0.666667,1198,0.575960,690.0,3,19,3,11.0,3,11.0,3,11.0,3,11.0,5.5,1.222222,2.500000,0
4629307,1,112108,4,10,9.0,0.666667,12,8.0,1.000000,12.0,7.0,2,11109,4.000000,8.0,2541372,0.666667,4472,0.713775,3192.0,108,16,3,11.0,3,11.0,3,11.0,3,11.0,2.0,0.777778,3.000000,1
4629304,1,112108,4,10,9.0,1.000000,12,8.0,1.333333,15.0,11.0,3,14947,3.666667,7.0,186706,0.666667,28884,0.812318,23463.0,115,7,3,11.0,3,11.0,3,11.0,3,11.0,5.0,1.222222,2.666667,0
4629302,1,112108,4,10,9.0,0.666667,12,8.0,1.000000,12.0,7.0,2,22035,5.000000,8.0,2541372,0.666667,59676,0.764780,45639.0,21,16,3,11.0,3,11.0,3,11.0,3,11.0,2.0,0.777778,3.000000,1


In [67]:
test_features.head()

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders_x,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,aisle_id,department_id,"(user_orders,)","('user_order_dsp',)_x","('user_product_orders',)_x","(user_order_dsp_y,)","(user_product_orders_y,)","('user_order_dsp',)_x","(user_product_orders,)","(user_order_dsp_y,)",diff_order_hod,ratio_dsp,diff_dow,user_product_hour_vs_last
858095,17,36855,6,15,1.0,0.25,21,5.0,0.000000,9.0,30.0,1,1283,5.000000,9.000000,234692,0.238095,29,0.206897,6.0,100,21,4,14.0,4,14.0,4,14.0,4,14.0,6.0,30.0,1.000000,3.0
858092,17,36855,6,15,1.0,0.25,21,5.0,6.000000,14.0,6.0,1,6291,9.000000,9.000000,3248434,0.238095,1968,0.618394,1217.0,121,14,4,14.0,4,14.0,4,14.0,4,14.0,1.0,6.0,0.000000,2.0
858088,17,36855,6,15,1.0,0.25,21,5.0,1.000000,13.0,NaN,1,7035,3.000000,5.000000,898818,0.238095,1933,0.478013,924.0,100,21,4,14.0,4,14.0,4,14.0,4,14.0,2.0,NaN,2.000000,2.0
858094,17,36855,6,15,1.0,0.25,21,5.0,1.000000,13.0,NaN,1,11494,5.000000,5.000000,898818,0.238095,1311,0.363082,476.0,100,21,4,14.0,4,14.0,4,14.0,4,14.0,2.0,NaN,2.000000,2.0
858097,17,36855,6,15,1.0,0.75,21,5.0,3.666667,12.0,14.0,3,13107,2.333333,7.333333,1058761,0.238095,373,0.635389,237.0,3,19,4,14.0,4,14.0,4,14.0,4,14.0,3.0,14.0,2.333333,3.0


In [69]:
train_orders = orders[orders['eval_set']=='train']
trainProducts = pandas.merge(train_orders,trainingOrderItems,on='order_id')
trainProducts.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [70]:
trainProducts = trainProducts.groupby('user_id')['product_id'].apply(set)
trainProducts.head(5)

user_id
1    {196, 26405, 27845, 46149, 13032, 39657, 26088...
2    {24838, 11913, 45066, 31883, 48523, 38547, 248...
5    {40706, 21413, 20843, 48204, 21616, 19057, 201...
7    {17638, 29894, 47272, 45066, 13198, 37999, 408...
8    {27104, 15937, 5539, 41540, 31717, 48230, 2224...
Name: product_id, dtype: object

In [71]:
trainLabels  =[]
for row in tqdm(train_features.itertuples()):
  trainLabels += [row.product_id in trainProducts[row.user_id]]

print(len(trainLabels))
print(train_features.shape)

8474661it [03:09, 44638.18it/s]

8474661
(8474661, 34)


In [0]:
num_feature_list=['diff_dow','ratio_dsp','diff_order_hod', 'user_avg_basket_size', 'user_product_hour_vs_last',
                  'days_since_prior_order','user_product_reorder_rate','user_product_dow','user_reordered_products',
                  'user_product_hod','user_product_dsp','user_product_orders','user_product_addCart',
                  'product_reorder_rate','product_total_orders', 'user_reorder_rate', 'user_distinct_products',
                  'user_product_orders_since_last']

In [74]:
train_features[num_feature_list] = (train_features[num_feature_list]-train_features[num_feature_list].mean())/((train_features[num_feature_list].max()-train_features[num_feature_list].min()))
test_features[num_feature_list] = (test_features[num_feature_list]-test_features[num_feature_list].mean())/((test_features[num_feature_list].max()-test_features[num_feature_list].min()))


KeyError: ignored

In [76]:
num_feature_list=['diff_dow','ratio_dsp','diff_order_hod', 'user_avg_basket_size', 'user_product_hour_vs_last',
                  'days_since_prior_order','user_product_reorder_rate','user_product_dow','user_reordered_products',
                  'user_product_hod','user_product_dsp','user_product_addCart',
                  'product_reorder_rate','product_total_orders', 'user_reorder_rate', 'user_distinct_products']

#num_feature_list=['diff_dow','ratio_dsp','diff_order_hod', 'user_avg_basket_size',
#                  'days_since_prior_order','user_product_reorder_rate','user_product_dow',
#                  'user_product_hod','user_product_dsp','user_product_addCart',
#                  'product_reorder_rate', 'user_reorder_rate', 
#                  'user_product_orders_since_last']
train_features[num_feature_list] = (train_features[num_feature_list]-train_features[num_feature_list].mean())/((train_features[num_feature_list].max()-train_features[num_feature_list].min()))
test_features[num_feature_list] = (test_features[num_feature_list]-test_features[num_feature_list].mean())/((test_features[num_feature_list].max()-test_features[num_feature_list].min()))

#cat_feature_list=['department_id','aisle_id','order_dow','order_hour_of_day']
cat_feature_list = []
feature_list = num_feature_list+cat_feature_list
test_features[num_feature_list].head()

,diff_dow,ratio_dsp,diff_order_hod,user_avg_basket_size,user_product_hour_vs_last,days_since_prior_order,user_product_reorder_rate,user_product_dow,user_reordered_products,user_product_hod,user_product_dsp,user_product_addCart,product_reorder_rate,product_total_orders,user_reorder_rate,user_distinct_products
858095,-0.170464,NaN,0.181576,-0.061448,-0.132405,-0.446237,0.096375,-0.458892,-0.14103,-0.196971,0.576189,-0.040674,-0.351908,-0.047506,-0.16614,-0.145215
858092,-0.456178,NaN,-0.235091,-0.061448,-0.215738,-0.446237,0.096375,0.541108,-0.14103,0.020420,-0.223811,-0.002212,0.090071,-0.043403,-0.16614,-0.145215
858088,0.115251,NaN,-0.151758,-0.098485,-0.215738,-0.446237,0.096375,-0.292225,-0.14103,-0.023058,NaN,-0.059905,-0.060709,-0.043477,-0.16614,-0.145215
858094,0.115251,NaN,-0.151758,-0.098485,-0.215738,-0.446237,0.096375,-0.292225,-0.14103,-0.023058,NaN,-0.040674,-0.184154,-0.044793,-0.16614,-0.145215
858097,0.210489,NaN,-0.068424,-0.076880,-0.132405,-0.446237,0.601477,0.152219,-0.14103,-0.066536,0.042856,-0.066315,0.108324,-0.046778,-0.16614,-0.145215


In [79]:
X_train = train_features[feature_list].fillna(0)[:500000]
Y_train = numpy.array(trainLabels).astype('int8')[:500000]
print (X_train[:10])
print(Y_train[:10])

         diff_dow  ratio_dsp  ...  user_reorder_rate  user_distinct_products
4629306  0.400300        0.0  ...           0.268539               -0.140862
4629308  0.257443        0.0  ...           0.268539               -0.140862
4629307  0.400300        0.0  ...           0.268539               -0.140862
4629304  0.305062        0.0  ...           0.268539               -0.140862
4629302  0.400300        0.0  ...           0.268539               -0.140862
4629301  0.400300        0.0  ...           0.268539               -0.140862
4629309  0.257443        0.0  ...           0.268539               -0.140862
4629310  0.400300        0.0  ...           0.268539               -0.140862
4629311  0.257443        0.0  ...           0.268539               -0.140862
4629303  0.400300        0.0  ...           0.268539               -0.140862

[10 rows x 16 columns]
[0 0 1 0 1 0 0 0 1 0]


In [83]:
numpy.random.seed(7)

model = Sequential()
model.add(Dense(13,input_dim=len(feature_list),activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam()
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=10,verbose=0,callbacks=[TQDMNotebookCallback()])

In [85]:
scores = model.evaluate(X_train,Y_train,verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


accuracy: 90.84%


In [0]:
X_cv = train_features[feature_list].fillna(0)[500000:1000000]
Y_cv = numpy.array(trainLabels).astype('int8')[500000:1000000]

In [87]:
preds = model.predict(X_cv)
preds.max()

0.99204636

In [88]:
print(preds[:10])
print(len(preds))

[[0.04692402]
 [0.03525957]
 [0.02015072]
 [0.31386697]
 [0.13350946]
 [0.15431646]
 [0.02668658]
 [0.02724305]
 [0.03756535]
 [0.0440391 ]]
500000


In [91]:
truePositive = 0
falsePositive = 0
falseNegative=0
trueNegative=0

def f_score(truePositive, falseNegative, falsePositive, beta=1):
    return ((1+beta**2)*truePositive)/(((1+beta**2)*truePositive)+beta**2*falseNegative+falsePositive)

for i in range(len(preds)):
    if preds[i] >.2:
        if (Y_cv)[i] ==1:
            truePositive +=1
        else:
            falsePositive+=1
    else:
        if (Y_cv)[i] ==1:
            falseNegative+=1
        else:
            trueNegative+=1

print (truePositive, trueNegative, falsePositive, falseNegative)
print (f_score(truePositive, falseNegative, falsePositive))

21036 415362 35953 27649
0.3981300982266215


In [0]:
X_test = test_features[feature_list].fillna(0)

In [0]:
test_preds = model.predict(X_test)

In [97]:
past_order_id = -1
reorderedProducts = []
output  = []
maxProd =[0,0]
for row in tqdm(test_features.itertuples()):
  if (row.order_id!=past_order_id):
    if (past_order_id == -1):
      pass
    else:
      if(reorderedProducts==[]):
        reorderedProducts.append(maxProd[0])
      output.append([past_order_id," ".join(reorderedProducts)])
      reorderedProducts = []
      maxProd= [0,0]
    past_order_id = row.order_id
  if (test_preds[i]>.2):
    reorderedProducts.append(str(row.product_id))
  else:
    if (test_preds[i] > maxProd[1]):
      maxProd  = [str(row.product_id),test_preds[i]]
  i+=1
output.append([past_order_id," ".join(reorderedProducts)])

4327771it [00:30, 143534.17it/s]

IndexError: ignored

In [98]:
output[5]

[313, '5322 17461 28465 28535 33738 40031 43352 45664']

In [0]:
with open('predictionNN.csv','w') as myfile:
  wr = csv.writer(myfile)
  wr.writerow(['order_id','products'])
  for row in output:
    wr.writerow(row)